+ The client first request the public key of asymmetric encryption from the server
+ Client generates a key for symmetric encryption
+ Client uses public key obtained from server to encrypt the public key and send to server
+ Server uses its own private key to decrypted the received the encrypted key from client
+ After the above negotiation process, server and client will use use symmetric encryption to encrypt the following communication with the negotiated key

In [9]:
import socket
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives import padding as pad
from cryptography.hazmat.primitives import hashes
import base64
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

In [10]:
# define a special request
class Request:
    # a simple request class containing method, url and headers
    def __init__(self, method: str, url: str, headers: {str: str}, request_data: str, body):
        self.method = method  # GET, POST, PUT, DELETE
        self.url = url  # e.g. /index.html
        self.headers = headers
        self.request_data = request_data
        self.body = body

    def convert(self):
        # output request
        # custom ENCRYPTION/1.1 request
        request = "{} {} ENCRYPTION/1.1\r\n".format(self.method, self.url)
        request += "\r\n".join("{}: {}".format(k, v) for k, v in self.headers.items())
        request += "\r\n\r\n" + self.body
        return request

In [11]:
# socket connect 127.0.0.1:8080/
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("127.0.0.1", 8080))

In [12]:
# first step: client request a public key
request = Request("GET", "/public_key", {}, "", "")
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break
public_key = serialization.load_pem_public_key(
    r,
    backend=default_backend()
)
print("public key size: ", public_key.key_size)

public key size:  2048


In [13]:
# second step: Client generates a key for symmetric encryption
symmetric_key = os.urandom(32)
print('Key Generated', symmetric_key)

iv = os.urandom(16)
print('IV Generated', iv)

Key Generated b'\x0e\xc0C\xe1\x8du\\Q\x91,\xa4\xe9\x85\xac?\xd8W!\xbf\x1d\xbe\x02\x05\xafYo\x11\x1a\\\x85\x8b\x89'
IV Generated b'J\x1ct\x8e\xb5\xecY\xecq+\xe3+ZMoR'


In [14]:
# third step: Client encrypts the symmetric key with the public key and sends it to the server

# encrypt the symmetric key with the public key
encrypted_key = public_key.encrypt(
    symmetric_key,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_symmetric_key = base64.b64encode(encrypted_key).decode("utf-8")
print('Key Encrypted', encrypted_symmetric_key)
print('Key Encrypted Length', len(encrypted_symmetric_key))

# send the encrypted symmetric key to the server
request = Request("POST", "/encrypted_symmetric_key", {"encrypted_symmetric_key": encrypted_symmetric_key}, "", "")
s.sendall(request.convert().encode("utf-8"))

encrypted_iv = public_key.encrypt(
    iv,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)
encrypted_iv = base64.b64encode(encrypted_iv).decode("utf-8")
print('IV Encrypted', encrypted_iv)
print('IV Encrypted Length', len(encrypted_iv))
request = Request("POST", "/encrypted_iv", {"encrypted_iv": encrypted_iv}, "", "")
s.sendall(request.convert().encode("utf-8"))

Key Encrypted RIBYIFBGskiCCwewDGXV2LddbzMKvFaNxOJaNjRYPbMxz9FG261dK/RW5eXnbwUs7gq7PjMj6UnYIV2V3oNmJGEAVCwPv2je1C0dcBz7kOIiwyq2p5caaRVAlfSdy0xYICVymCQhdsCxZz+g1xO94CXUDJtOuS947dR7x24+huCleTAxJLQBgca6rPVmSpkjFPO8X7tHGbzJXYQB10VpVBesA6WbshELH5I8Y+QZrq68PlSrfVVbcM4x5koJz+AhzzSusZDBL6PTxvfnTU8sKlVtpQWAl1Sid7lilZvZX4/TXBAiUe/ZNAm2VAAJQphxwvipFjoevukB5Jjcz2uY/A==
Key Encrypted Length 344
IV Encrypted R1pqMCDi3vvFPTym3OQVRg3UsrgmwLGOwh9dED3tZU002mlsm2t1J+ZF4GWrMlFXXCxfwIgUx5sIGYo0apKpHbm/d3TQ9yin6F8ffHkhtdTrQqIPDe2PGB1Yv+9YuzFrIx+ncs8f4dh4RyVmWAWNOX5UGoAVFPilSZFXBM9TcF8qKsgBKeS9uYbz5ChMooSnUd/+vyOAFatNi+RUL2P/rNYhkSqxBVhONOpR6U9peKGZiHiFxkwLHNbEHxlgR35KNpbBWnRwsSjmQEDNOvWmEJV7ZukMYVurmqVT7m8rqJBYBaky8aVvaQpDQF/h2BGSZdgdRJdjqrVgkJaMR0mwuw==
IV Encrypted Length 344


In [17]:
body = """------WebKitFormBoundary7MA4YWxkTrZu0gW\r
Content-Disposition: form-data; name="firstfile"; filename="a.txt"\r
Content-Type: text/plain\r
\r
I love U 2!\r
------WebKitFormBoundary7MA4YWxkTrZu0gW--"""

# fourth step: Client encrypts the file with the symmetric key and sends it to the server
# encrypt the file with the symmetric key
cipher = Cipher(algorithms.AES(symmetric_key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()

# Pad the body to ensure it is a multiple of the block length
block_size = cipher.algorithm.block_size // 8
padder = pad.PKCS7(block_size * 8).padder()
padded_body = padder.update(body.encode('utf-8')) + padder.finalize()

encrypted_body = encryptor.update(padded_body) + encryptor.finalize()
encrypted_body = base64.b64encode(encrypted_body).decode("utf-8")
print('Body Encrypted', encrypted_body)

headers = {
    "Content-Type": "multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW",
    "Content-Length": len(encrypted_body),
    "Authorization": "Basic Y2xpZW50MToxMjM="
}

request = Request("POST", "/upload?path=client1/", headers, "", encrypted_body)
print(request.convert())
s.sendall(request.convert().encode("utf-8"))
while True:
    r = s.recv(1024)
    if r is not None:
        break

print("\n")
print(r.decode("utf-8"))

Body Encrypted DAgKow7RdIgKScaj6wPR9oi2zPqbkKyHkVBZ7/kQeMeq3Tccri1Tr3s9KB+f1/i2IGCekC8qTxAgF9acykVdc7a4I52RqNhUQpuCK3vzfdwbXt2OO/J+cYt8cSDBbarY7TfwItulIw231qp0gqBN1m4JULxXpLMghJ+yAgFySjnP6Z6R3YMEVzC5bmrUEMk3e8G8PTa0KulDoH/wz0Eq8EAxSuuFIAqyh9bR3JeOGiKcTLJz9ylrUVHtMwSXi16p
POST /upload?path=client1/ ENCRYPTION/1.1
Content-Type: multipart/form-data; boundary=----WebKitFormBoundary7MA4YWxkTrZu0gW
Content-Length: 256
Authorization: Basic Y2xpZW50MToxMjM=

DAgKow7RdIgKScaj6wPR9oi2zPqbkKyHkVBZ7/kQeMeq3Tccri1Tr3s9KB+f1/i2IGCekC8qTxAgF9acykVdc7a4I52RqNhUQpuCK3vzfdwbXt2OO/J+cYt8cSDBbarY7TfwItulIw231qp0gqBN1m4JULxXpLMghJ+yAgFySjnP6Z6R3YMEVzC5bmrUEMk3e8G8PTa0KulDoH/wz0Eq8EAxSuuFIAqyh9bR3JeOGiKcTLJz9ylrUVHtMwSXi16p


HTTP/1.1 200 OK
Connection: keep-alive
Content-Type: text/html
Set-Cookie: session-id=b'\xb4\xf6>\xf5\xde\x1a\xb2\xed\xeb\xb9M!\xe2\x89\xc8O\xcd\x8f\x8b\xc9\xefe$\x80'
Content-Length: 44

mkf9bb9jvaY5+zLw0lZ0vb1yqHSayf8YoZI7mu8bZFo=


In [22]:
# get body from response
r = r.decode("utf-8").split("\r\n\r\n")[1]

# decrypt the response using the symmetric key and iv
decryptor = cipher.decryptor()
decrypted_body = decryptor.update(base64.b64decode(r)) + decryptor.finalize()
unpadder = pad.PKCS7(block_size * 8).unpadder()
decrypted_body = unpadder.update(decrypted_body) + unpadder.finalize()

print(decrypted_body.decode("utf-8"))

encrypted response
